In [30]:
from dataloader import DataLoader


trainset = DataLoader.load_data("../data/train")
devset = DataLoader.load_data("../data/dev")

print(f"{trainset[:3]=}")
print(f"{devset[:3]=}")


trainset[:3]=[Record(original='Title: Review iPhone 14: Thiết kế, hiệu năng, camera và pin', sentence='Title : Review iPhone 14 : Thiết kế , hiệu năng , camera và pin', quintuples=[], comparative=['No']), Record(original='iPhone 14 được ra mắt vào 09/2022, được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước.', sentence='iPhone 14 được ra mắt vào 09 / 2022 , được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước .', quintuples=[{'subject': ['1&&iPhone', '2&&14'], 'object': ['28&&thế', '29&&hệ', '30&&trước'], 'aspect': ['20&&phần', '21&&cứng'], 'predicate': ['16&&nâng', '17&&cấp', '18&&nhẹ'], 'label': 'COM+'}, {'subject': ['1&&iPhone', '2&&14'], 'object': ['28&&thế', '29&&hệ', '30&&trước'], 'aspect': ['24&&phần', '25&&mềm'], 'predicate': ['16&&nâng', '17&&cấp', '18&&nhẹ'], 'label': 'COM+'}], comparative=['COM+', 'COM+']), Record(original='Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm.', sentence='Về n

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()

sentences = [record.sentence for record in trainset]

embeded_sentences = tfidf_vec.fit_transform(sentences)

print(embeded_sentences.shape)

(4171, 2464)


In [32]:
from sklearn.preprocessing import MultiLabelBinarizer

labels = [record.comparative for record in trainset]
label_classes = ["No", "DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]

multilabel_model = MultiLabelBinarizer(classes=label_classes)
embeded_labels = multilabel_model.fit_transform(labels)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler

x_train, x_val, y_train, y_val = train_test_split(embeded_sentences, embeded_labels, test_size=0.2)


In [34]:
print(f"{x_train.shape=}")
print(f"{x_train.shape=}")
print(embeded_labels[:10])

x_train.shape=(3336, 2464)
x_train.shape=(3336, 2464)
[[1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]


In [35]:
from sklearn.multioutput import MultiOutputClassifier

randomforest_model = RandomForestClassifier(n_estimators = 500, criterion = 'entropy',max_depth = 10, class_weight='balanced')
multioutput_classifier = MultiOutputClassifier(randomforest_model)
multioutput_classifier.fit(x_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       criterion='entropy',
                                                       max_depth=10,
                                                       n_estimators=500))

In [36]:
y_preds = multioutput_classifier.predict(x_val)

print('Accuracy = ', accuracy_score(y_val, y_preds))
report = classification_report(y_val, y_preds)
print(report)

Accuracy =  0.7832335329341318
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       676
           1       0.00      0.00      0.00         9
           2       0.61      0.44      0.51        52
           3       1.00      0.21      0.35        19
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.68      0.67      0.68        73
           7       1.00      0.05      0.10        19
           8       0.00      0.00      0.00         5

   micro avg       0.90      0.79      0.84       856
   macro avg       0.47      0.25      0.28       856
weighted avg       0.88      0.79      0.82       856
 samples avg       0.81      0.80      0.80       856



/home/ppvan/Documents/code/academic/data-mining/datamine/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppvan/Documents/code/academic/data-mining/datamine/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
sample_sentence = "Dù là chip \" xịn xò \" nhất so với các điện thoại Android nhưng khi so sánh với iPhone iPhone 13 Pro Max có phần chiếm ưu thế hơn ."
embeded_sentence = tfidf_vec.transform([sample_sentence])
prediction = multioutput_classifier.predict(embeded_sentence)
print(multilabel_model.inverse_transform(prediction))
print(prediction)

[('COM+',)]
[[0 0 0 0 0 0 1 0 0]]


## Task 2: Extract đầy đủ cái đặc trưng của phép so sánh.

**Phương pháp** : Conditional Random Fields (CRF)

Input là sentence

Output là POS-tagging

--> Xác định được POS-tag của từng từ trong câu

--> Xác định được entity(subject, object) ,aspect, predicate

**Subject trùng với Object --> Hết cứu**

In [38]:
tag = ['B-Sub','I-Sub','B-Obj','I-Obj','B-Asp','I-Asp','B-Pre','I-Pre']
special_characters = [',', '.', ':', ';', '?', '!', '"', "'", '(', ')', '-', '/', '\\', '*', '%', '#', '@', '&', '_', '[', ']', '{', '}', '|', '<', '>', '`', '~', '^', '+', '=', '|', '/', '?', ';',':', '“','”']

In [39]:
from sklearn_crfsuite import CRF


crf = CRF(
    algorithm="lbfgs",
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)


In [40]:
from processing import RecordEncoder
from utils import dupplicate_record_by_quintuple

cleaned_trainset = []
for record in trainset:
    if not record.quintuples:
        continue

    cleaned_trainset.extend(dupplicate_record_by_quintuple(record))

encoder = RecordEncoder()


x = [encoder.record2features(record) for record in clean_trainset]
y = [encoder.record2tags(record) for record in clean_trainset]


crf.fit(x, y)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

### Predict cho task 1: Liệu câu có so sánh hay không?

In [41]:
dev_embeded_sentences = tfidf_vec.transform([record.sentence for record in devset])

print(dev_embeded_sentences.shape)

comparative_preditions = multioutput_classifier.predict(embeded_sentences)
comparative_preditions.shape


(1733, 2464)


(4171, 9)

Decode vector về dạng string

Trong trường hợp model không đưa ra label nào, ta mặc định nó không có câu so sánh ("No")

In [43]:
readable_comparative_preditions = multilabel_model.inverse_transform(comparative_preditions)

formatted_comparative_predictions = []

for prediction in readable_comparative_preditions:
    if prediction:
        formatted_predictions.append(list(prediction))
    else:
        formatted_predictions.append(["No"])

formatted_comparative_predictions[:5]

[]

### Predict cho task 2: Chi tiết các vật thể được so sánh

In [44]:
from processing import RecordDecoder

dev_featured_sentences = [encoder.record2features(record) for record in devset]
decoder = RecordDecoder()


postag_preditions = crf.predict(dev_featured_sentences)

readable_postag_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in postag_preditions]

readable_postag_predictions[0]


['O',
 ':',
 'O',
 'O',
 'B-Sub',
 'I-Sub',
 'I-Sub',
 'I-Sub',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '.']

In [29]:

labeled_records = [
    decoder.label_record(record, formatted_predictions[i], readable_postag_predictions[i])
    for i, record in enumerate(devset)]

for record in labeled_records:
    
    if len(record.quintuples) > 0:
        print(record.quintuples)



[{'label': 'No', 'subject': ['5&&iPhone', '6&&13', '7&&Pro', '8&&Max'], 'object': [], 'aspect': [], 'predicate': []}]
[{'label': 'COM+', 'subject': ['1&&iPhone', '2&&13', '3&&Pro', '4&&Max', '6&&Samsung', '7&&Galaxy', '8&&Z', '9&&Fold3'], 'object': [], 'aspect': [], 'predicate': ['13&&chiến', '14&&mã']}]
[{'label': 'EQL', 'subject': [], 'object': [], 'aspect': [], 'predicate': []}]
[{'label': 'No', 'subject': [], 'object': [], 'aspect': [], 'predicate': []}]
[{'label': 'No', 'subject': [], 'object': [], 'aspect': ['1&&Thiết', '2&&kế'], 'predicate': []}]
[{'label': 'No', 'subject': [], 'object': ['17&&hai', '18&&chiếc', '19&&điện', '20&&thoại'], 'aspect': ['2&&thiết', '3&&kế'], 'predicate': []}]
[{'label': 'No', 'subject': [], 'object': [], 'aspect': [], 'predicate': ['19&&rất', '20&&sang', '21&&trọng']}]
[{'label': 'No', 'subject': [], 'object': [], 'aspect': [], 'predicate': []}]
[{'label': 'No', 'subject': ['8&&Samsung', '9&&Galaxy', '10&&Z', '11&&Fold3'], 'object': [], 'aspect': [],